# Daily Blog #100 - Stack and Function Calls
### August 8, 2025 

---

## **1. The Call Stack: Conceptual Overview**

The **call stack** is a special area of memory used to manage **function calls** and **returns**. It operates in a **last-in, first-out (LIFO)** fashion and is primarily used to:

* Pass arguments to functions
* Store return addresses
* Save the state of registers (especially caller-saved ones)
* Allocate space for local variables

Each function call creates a **stack frame**, which contains all the necessary information to resume the caller function once the callee finishes execution.

---

## **2. Stack Basics**

### **2.1 Stack Growth Direction**

* On most architectures (including MIPS and x86), the stack grows **downward**—toward **lower memory addresses**.
* A special register called the **Stack Pointer (SP)** always points to the **top of the stack**.

### **2.2 Key Operations**

* `PUSH`: Decrement SP and write a value to the new top of the stack
* `POP`: Read value from top of the stack and increment SP

> In MIPS, there are no native `push` or `pop` instructions; the stack is manipulated manually using `sw` (store word) and `lw` (load word), along with `addi` to adjust the SP.

---

## **3. Stack in MIPS Assembly**

### **3.1. Important Registers**

| Register       | Role                     |
| -------------- | ------------------------ |
| `$sp`          | Stack Pointer            |
| `$ra`          | Return Address           |
| `$fp` or `$s8` | Frame Pointer (optional) |

### **3.2. Function Call Steps in MIPS**

#### Caller Side:

1. Save caller-saved registers (e.g., `$t0-$t9`) on the stack
2. Pass arguments via `$a0-$a3` (or stack if more than 4)
3. Use `jal function_label` to jump and save return address in `$ra`

#### Callee Side:

1. Allocate space on the stack for local variables
2. Save `$ra` and any callee-saved registers (`$s0-$s7`) if used
3. Function body runs
4. Restore registers, deallocate stack space
5. Return using `jr $ra`

### **3.3. MIPS Example**

```assembly
# Caller
addi $sp, $sp, -4
sw $t0, 0($sp)        # Save $t0
li $a0, 5
jal my_function
lw $t0, 0($sp)        # Restore $t0
addi $sp, $sp, 4

# Callee
my_function:
  addi $sp, $sp, -8
  sw $ra, 4($sp)       # Save return address
  sw $s0, 0($sp)       # Save callee-saved register

  move $s0, $a0        # Function logic
  # ...
  
  lw $s0, 0($sp)
  lw $ra, 4($sp)
  addi $sp, $sp, 8
  jr $ra
```

---

## **4. Stack in x86 Assembly (Intel Syntax)**

### **4.1. Important Registers**

| Register           | Role                                          |
| ------------------ | --------------------------------------------- |
| `ESP`              | Stack Pointer                                 |
| `EBP`              | Frame Pointer                                 |
| `EIP`              | Instruction Pointer (not directly accessible) |
| `EAX`, `EBX`, etc. | General purpose                               |

### **4.2. Function Call Steps in x86**

#### Caller Side:

1. Push arguments **right to left**
2. Use `CALL function_name`
3. After return, clean arguments off the stack (if using `cdecl` convention)

#### Callee Side:

1. Prologue:

   * `PUSH EBP`
   * `MOV EBP, ESP`
   * `SUB ESP, n` (space for locals)
2. Function body
3. Epilogue:

   * `MOV ESP, EBP`
   * `POP EBP`
   * `RET`

### **4.3. x86 Example**

```assembly
; Caller
PUSH 5               ; Argument
CALL my_function
ADD ESP, 4           ; Clean up argument

; Callee
my_function:
  PUSH EBP
  MOV EBP, ESP
  SUB ESP, 4         ; Local variable space

  MOV EAX, [EBP+8]   ; Access argument (5)
  ; Function body here

  MOV ESP, EBP
  POP EBP
  RET
```

> In this example, `[EBP+8]` accesses the first argument, `[EBP-4]` would access a local variable.

---

## **5. Calling Conventions**

Calling conventions define **how functions pass arguments**, **return values**, and **preserve registers**. They are essential for compatibility across compilers and libraries.

### **5.1. MIPS Convention Summary**

* **Arguments**: `$a0` to `$a3`, rest on stack
* **Return values**: `$v0` (and `$v1` for 64-bit)
* **Caller-saved**: `$t0`–`$t9`
* **Callee-saved**: `$s0`–`$s7`, `$ra`

### **5.2. x86 (cdecl) Convention Summary**

* **Arguments**: Passed on the stack (right to left)
* **Return value**: In `EAX`
* **Caller-saved**: `EAX`, `ECX`, `EDX`
* **Callee-saved**: `EBX`, `ESI`, `EDI`, `EBP`

---

## **6. Stack Frame Layout (General)**

```
High Address
---------------
Previous EBP (Frame Pointer)
Return Address
Function Arguments
---------------
Local Variables
Temporary/Spilled Registers
---------------
Low Address (ESP)
```

Each function gets its own stack frame. Once a function returns, its frame is destroyed by popping off the stack.

---

## **7. Recursive Function Calls**

The stack allows **recursion** because each function call pushes its own return address, local variables, and arguments. When a recursive call is made, a **new stack frame** is created, separate from the previous one.

Example: Computing factorial recursively

```c
int fact(int n) {
  if (n == 1) return 1;
  else return n * fact(n - 1);
}
```

Each recursive call pushes a new `n` and return address onto the stack, enabling correct backtracking.

---

## **8. Stack Overflows and Buffer Overflows**

* **Stack Overflow**: Happens when too many function calls or too large stack allocations exceed memory bounds.
* **Buffer Overflow**: Occurs when writing outside allocated space, often overwriting return addresses—exploited in attacks.

Proper management of stack operations is critical for **security** and **stability**.

---

## **9. Summary: MIPS vs. x86 Stack and Calls**

| Feature                 | MIPS                   | x86 (Intel Syntax)         |
| ----------------------- | ---------------------- | -------------------------- |
| Stack Growth            | Downward               | Downward                   |
| Arguments Passed        | `$a0-$a3`, then stack  | Stack (right to left)      |
| Return Value Register   | `$v0`, `$v1`           | `EAX`                      |
| Return Instruction      | `jr $ra`               | `RET`                      |
| Stack Manipulation      | Manual with `sw`, `lw` | `PUSH`, `POP` instructions |
| Calling Convention Type | Register-first         | Stack-based                |
